## Sztuczne sieci neuronowe - laboratorium 8

### Splotowe sieci neuronowe - cz. 2

Na poprzednich zajęciach poznaliśmy warstwy tworzące **splotową sieć neuronową** i nauczyliśmy się tworzyć modele jako klasy  dziedziczące po `nn.Module`.

Dziś wytrenujemy splotową sieć neuronową do binarnej klasyfikacji obrazu i poznamy dodatkowe techniki stosowane w sieciach neuronowych, m.in. do regularyzacji modeli.

#### Pytania kontrolne

1. Opisz budowę splotowej sieci neuronowej. Wyjaśnij, do czego służą jej poszczególne warstwy.
2. Na czym polega regularyzacja modeli?
3. Jakie znasz metody regularyzacji stosowane w sieciach neuronowych?

### Z poprzednich ćwiczeń

Uruchom kolejne komórki, wykorzystujące kod z poprzednich zajęć, aby przygotować zbiór danych - `cifar2` oraz klasę `Net` definiującą model.

In [11]:
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
class_names = {
    0: "airplane", 
    1: "automobile", 
    2: "bird", 
    3: "cat", 
    4: "deer", 
    5: "dog", 
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}

In [3]:
tensor_cifar10 = datasets.CIFAR10("data", train=True, download=False, transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10("data", train=False, download=False, transform=transforms.ToTensor())

In [4]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
per_channel_means = imgs.view(3, -1).mean(dim=1)
per_channel_std = imgs.view(3, -1).std(dim=1)

In [5]:
transforms_compose = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(per_channel_means, per_channel_std)
])

transformed_cifar10 = datasets.CIFAR10("data", train=True, download=False, transform=transforms_compose)
transformed_cifar10_val = datasets.CIFAR10("data", train=False, download=False, transform=transforms_compose)

In [6]:
label_map = {0: 0, 2: 1}
new_class_names  = [class_names[i] for i in label_map]

cifar2 = [(img, label_map[label]) for img, label in tensor_cifar10 if label in label_map]
cifar2_val = [(img, label_map[label]) for img, label in tensor_cifar10 if label in label_map]

In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
    
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

### Trening modelu

#### Ćwiczenie
Uzupełnij poniższe komórki, aby wytrenować splotową sieć neuronową do zadania klasyfikacji binarnej.

Przyjmij learning rate o wartości 0.01 i batch size 64. Trenuj przez 100 epok. Użyj optymaliatora SGD.

In [37]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = loss_fn(outputs, labels.long())
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train}")

AttributeError: 'list' object has no attribute 'shape'

In [38]:
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
train_loader = DataLoader(cifar2, batch_size=64, shuffle=True)

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 105.95751374959946
Epoch 11, Training loss 65.94430935382843
Epoch 21, Training loss 55.472624346613884
Epoch 31, Training loss 51.47982883453369
Epoch 41, Training loss 48.44425129890442
Epoch 51, Training loss 45.559572488069534
Epoch 61, Training loss 43.25438152253628
Epoch 71, Training loss 41.051490783691406
Epoch 81, Training loss 39.26327306777239
Epoch 91, Training loss 36.70376615971327


### Walidacja modelu

#### Ćwiczenie

Zaimplementuj funkcję `validate`, która zmierzy dokładność wytrenowanego modelu na dwóch zbiorach - uczącym i walidacyjnym.

Porównaj wyniki z wynikami z laboratorium nr 5 (sieć gęsta).

In [39]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

In [42]:
def validate(model, train_loader, val_loader):
    model.eval()
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
        print(f"{name} accuracy: {correct/total}")

In [43]:
validate(model, train_loader, val_loader)

train accuracy: 0.9033
val accuracy: 0.9033


### Zapis i odczyt modelu

Wytrenowany model (zwłaszcza tak, którego trening trwa długo) warto zapisać, aby móc go użyć później.

Typowo po każdej epoce treningu sprawdza się działanie modelu na zbiorze walidacyjnym (walidacja).
Zapisu modelu (tzw. "checkpoint") typowo dokonuje się, jeśli wartość danej metryki (np. dokładność lub F1 na zbiorze walidacyjnym) jest lepsza niż najlepsza uzyskana dotychczas.

PyTorch pozwala zapisać wagi (parametry) modelu z użyciem `torch.save` oraz tzw. `state_dict` modelu (https://pytorch.org/tutorials/recipes/recipes/what_is_state_dict.html). Innym sposobem zapisu jest zapis całego modelu (z użyciem `pickle` "pod spodem"):
https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html

Następnie, do wczytania zapisanego modelu można użyć metody `load_state_dict` (oraz `torch.load`), jeśli zapisywaliśmy tylko `state_dict` lub tylko `torch.load`, jeśli zapisywaliśmy cały model.

In [44]:
print(model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[ 0.1399, -0.2607, -0.0720],
          [-0.2648, -0.3619, -0.2403],
          [-0.2758, -0.3072, -0.3164]],

         [[ 0.0725,  0.1068,  0.2232],
          [ 0.0172, -0.0714,  0.0909],
          [ 0.0788, -0.1914, -0.2757]],

         [[ 0.3221,  0.3741,  0.0511],
          [ 0.0111,  0.2005,  0.2469],
          [ 0.0897, -0.0769,  0.2289]]],


        [[[-0.0833,  0.0216,  0.0188],
          [ 0.2041, -0.0078, -0.3790],
          [ 0.0608, -0.0866, -0.2872]],

         [[-0.0092,  0.1148, -0.0330],
          [ 0.2250,  0.1591, -0.2070],
          [ 0.2663,  0.1208,  0.0754]],

         [[ 0.2024, -0.1487, -0.3180],
          [ 0.3235, -0.0661, -0.3352],
          [ 0.2288, -0.1104, -0.2788]]],


        [[[ 0.2502,  0.2309,  0.2779],
          [ 0.3580,  0.2259, -0.0228],
          [-0.0776,  0.0790, -0.0452]],

         [[-0.2362, -0.1300, -0.2186],
          [-0.1832,  0.0413, -0.2576],
          [-0.1222, -0.2921, -0.2645]],

         [[ 0.

In [45]:
torch.save(model.state_dict(), "data/birds_vs_airplanes.pt")

In [46]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load("data/birds_vs_airplanes.pt"))

<All keys matched successfully>

### Trening na GPU (opcjonalnie)

Aby przyspieszyć trening (zwłaszcza w przypadku głębokich modeli i dużych zbiorów danych), powszechnie stosuje się karty graficzne (GPU). Jeśli mamy dostęp do maszyny z kartą graficzną (najlepiej od NVIDIA, obsługującą CUDA), możemy łatwo "przenieść" trening na GPU.

W tym celu należy przenieść zarówno dane, jak i model, na kartę graficzną, używając metody `.to` (tensora i `nn.Module`) na zdefiniowane urządzenie (patrz poniżej).

In [47]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on {device}")

Training on cuda


#### Ćwiczenie

Zmodyfikuj napisaną wyżej pętlę treningową oraz inicjalizację modelu, przenosząc odpowiednio dane (obrazki i etykiety) oraz model na `device`.

In [48]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = loss_fn(outputs, labels.long())
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train / len(train_loader)}")

In [50]:
model_gpu = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_gpu.parameters(), lr=0.01)
train_loader = DataLoader(cifar2, batch_size=64, shuffle=True)

training_loop(n_epochs=100, optimizer=optimizer, model=model_gpu, loss_fn=loss_fn, train_loader=train_loader)

Epoch 1, Training loss 0.683061994564761
Epoch 10, Training loss 0.472398551026727
Epoch 20, Training loss 0.3635449573682372
Epoch 30, Training loss 0.33564495651205634
Epoch 40, Training loss 0.30892097000863145
Epoch 50, Training loss 0.28991292986520534
Epoch 60, Training loss 0.2725954226627471
Epoch 70, Training loss 0.2589002258268891
Epoch 80, Training loss 0.24738430929411748
Epoch 90, Training loss 0.23359656447817564
Epoch 100, Training loss 0.22354738153279965


### Rozbudowa modelu

Możemy "powiększyć" model "na szerokość" (dodać więcej filtrów) lub "na głębokość" (dodać więcej warstw).

#### Ćwiczenie

Zmodyfikuj klasę `Net` i stwórz kolejno:
- `NetWidth` - 2x więcej filtrów w warstwach splotowych (niech liczba filtrów będzie argumentem konstruktora)
- `NetDepth` - dodatkowa warstwa splotowa `conv3`

Pamiętaj o zmodyfikowaniu metody `forward`.

In [51]:
class NetWidth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1*2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1*2, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = nn.functional.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = nn.functional.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans*2)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [52]:
class NetDepth(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = nn.functional.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = nn.functional.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = nn.functional.max_pool2d(torch.relu(self.conv3(out)), 2)
        out = out.view(-1, 8 * 8 * self.n_chans)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

### Regularyzacja L2 / weight decay

Regularyzację L2 można zaimplementować samemu (jak niżej). 

Jest ona jednak wbudowana w `torch.optim`, np. https://pytorch.org/docs/stable/optim.html#torch.optim.SGD, gdzie wystarczy podać wartość `weight_decay` tworząc optymalizator.

#### Ćwiczenie

W poniższej pętli treningowej dopisz fragment realizujący regularyzację L2 dla lambda = 0.001.

In [60]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device, dtype=torch.float32)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
                        
            l2_lambda = 0.001
            for param in model.parameters():
                loss += l2_lambda * torch.norm(param)**2

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train}")

In [61]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

Epoch 1, Training loss 110.4469341635704
Epoch 10, Training loss 75.77257165312767
Epoch 20, Training loss 62.245709985494614
Epoch 30, Training loss 55.95564502477646
Epoch 40, Training loss 53.32600408792496
Epoch 50, Training loss 50.80273789167404
Epoch 60, Training loss 48.787751004099846
Epoch 70, Training loss 47.15585224330425
Epoch 80, Training loss 45.417018070816994
Epoch 90, Training loss 44.2782152146101
Epoch 100, Training loss 42.82187984883785


#### Ćwiczenie

Wywołaj "zwykłą" pętlę treningową, tym razem podająć `weight_decay` optyamlizatora równe 0.001. Zaobserwuj wpływ na funkcję straty.

### Dropout

https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html

#### Ćwiczenie
Dołóż warstwy `nn.Dropout2d` po warstwach splotowych (po max poolingu) do sieci `Net` i stwórz w ten sposob `NetDropout`.

In [ ]:
class NetDropout(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)
        self.dropout = nn.Dropout2d(p=0.5)

    def forward(self, x):
        out = nn.functional.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.dropout(out)
        out = nn.functional.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = self.dropout(out)
        out = out.view(-1, 8 * 8 * self.n_chans)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

### Batch normalization

https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

#### Ćwiczenie

Dodaj warstwy `BatchNorm2d` po warstwach splotowych `Net` tworząc w ten sposób `NetBatchNorm`.

In [ ]:
class NetBatchNorm(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()        
        self.n_chans = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(n_chans1)
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = nn.functional.max_pool2d(torch.relu(self.bn1(self.conv1(x))), 2)
        out = nn.functional.max_pool2d(torch.relu(self.bn2(self.conv2(out))), 2)
        out = out.view(-1, 8 * 8 * self.n_chans)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

#### Ćwiczenie (zadanie domowe)

Porównaj wyniki uzyskane przez każdą z zaimplementowanych klas na zbiorze uczącym i walidacyjnym.

In [62]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
loss_fn = nn.CrossEntropyLoss()
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)

model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)
print(f"Walidacja modelu: {model.type()}")
validate(model, train_loader, val_loader)


model_NetWidth= NetWidth().to(device)
optimizer = torch.optim.SGD(model_NetWidth.parameters(), lr=1e-2)
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model_NetWidth,
    loss_fn = loss_fn,
    train_loader = train_loader
)
print(f"Walidacja modelu: {model_NetWidth.type()}")
validate(model_NetWidth, train_loader, val_loader)


model_NetDepth = NetDepth().to(device)
optimizer = torch.optim.SGD(model_NetDepth.parameters(), lr=1e-2)
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model_NetDepth,
    loss_fn = loss_fn,
    train_loader = train_loader
)
print(f"Walidacja modelu: {model_NetDepth.type()}")
validate(model_NetDepth, train_loader, val_loader)


model_NetDropout = NetDropout().to(device)
optimizer = torch.optim.SGD(model_NetDropout.parameters(), lr=1e-2)
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model_NetDropout,
    loss_fn = loss_fn,
    train_loader = train_loader
)
print(f"Walidacja modelu: {model_NetDropout.type()}")
validate(model_NetDropout, train_loader, val_loader)

model_NetBatchNorm = NetBatchNorm().to(device)
optimizer = torch.optim.SGD(model_NetBatchNorm.parameters(), lr=1e-2)
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model_NetBatchNorm,
    loss_fn = loss_fn,
    train_loader = train_loader
)
print(f"Walidacja modelu: {model_NetBatchNorm.type()}")
validate(model_NetBatchNorm, train_loader, val_loader)

Epoch 1, Training loss 111.15281909704208
Epoch 10, Training loss 72.56290775537491
Epoch 20, Training loss 60.283688858151436
Epoch 30, Training loss 56.228682935237885
Epoch 40, Training loss 53.87071466445923
Epoch 50, Training loss 52.15117555856705
Epoch 60, Training loss 50.45057292282581


KeyboardInterrupt: 